## Imports

In [1]:
# all the necessary imports to run the file
# https://jakevdp.github.io/blog/2017/12/05/installing-python-packages-from-jupyter/
import sys

# installing packages that may not be installed on user's computer
!{sys.executable} -m pip install sounddevice
!{sys.executable} -m pip install librosa
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install keras

import os
import sounddevice as sd
from scipy.io.wavfile import write
import librosa
from pathlib import Path
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
import time
import shutil
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore') #don't display warnings about librosa falling back onto another package


/opt/anaconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/opt/anaconda3/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
Using TensorFlow backend.


## Initial Data Processing

<pr> The section does NOT need to be run when using the program. It was used to initially load in all the audio files, take 20 mfccs, transform the data, and save the data as numpy arrays for future use. <pr>

In [ ]:
X = list()
y = list()
max_padding = 0

data = Path('.gitignore/tone_perfect_mp3').iterdir()

for item in data:
    if item.is_file():
        filename = item.name.split("_") #"{}{}_{}V{}_MP3.mp3".format(mword, tone, gender, personID)
        y.append(int(filename[0][-1])) 
        
        #transform all of the data to mfccs (Mel Frequency Cepstral Coefficients)
        #https://medium.com/prathena/the-dummys-guide-to-mfcc-aceab2450fd
        
        audio, sampling_rate = librosa.load(item)
        mfcc = librosa.feature.mfcc(y=audio, sr=sampling_rate, n_mfcc=20) 
        X.append(mfcc)
        total_audio_time = len(mfcc[0])
        if total_audio_time > max_padding: max_padding = total_audio_time

#padding each audio file's mcffs with zero so all audio files are considered the same duration 
for i in range(len(X)):
    X[i] = pad_sequences(X[i], padding='post', maxlen = max_padding)

#transforming from (file, coefficients, time) to (file, time, coefficients)
X = np.asarray(X, dtype=np.float32)
X = np.transpose(X, (0, 2, 1))

In [ ]:
#save the numpy arrays to significatly cut down on run time 
np.save('mcffs.npy', X)
np.save('labels.npy', y)

## Loading in Data

In [2]:
X = np.load('mcffs.npy')
y = np.load('labels.npy')

# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state=42)

In [ ]:
# Possibly need to split the data into M vs F users
# in this case, we would have to split the data into male or female and
# train 2 different models and predict on both those models and find the 
# the closest one OR have the person
#
# for sex in ["MV", "FV"]:
#     for label in range(1,5):
#         try:
#             directory_name = ".gitignore/{}{}".format(sex,label)
#             os.makedirs(directory_name)    
#             print("Directory " , directory_name ,  " Created ")
#         except FileExistsError:
#             print("Directory " , directory_name ,  " already exists") 

In [ ]:
# principal component analysis?

# User Interface

## Create folder to hold user audio
<p> Run the following cell to remove previous user audio and make the folder to hold all user input audio. <p>

In [ ]:
#initializing folder to hold user audio
user_audio_directory = ("user_audio")
try:
    shutil.rmtree(user_audio_directory)
    os.makedirs(user_audio_directory)
    print("Removed data and recreated folder", user_audio_directory)
except(FileNotFoundError):
    os.makedirs(user_audio_directory)
    print("Created folder {}".format(user_audio_directory)) 

## Record user's voice

<pr> Following cell records a user's voice. Users are encouraged to only record monosyllabic words in order to practice tonalities. An unclear recording will result in a low<pr>

In [ ]:
frame_rate = 44100
duration = 1.5 #seconds

print("Recording in...")
pause = 3

while pause > 0:
    print("{}".format(pause))
    pause -= 1
    time.sleep(1)
print("RECORDING")

recording = sd.rec(int(duration * frame_rate), samplerate = frame_rate, channels = 1)
recording_num = len(os.listdir('user_audio/')) + 1
recording_name = 'user_audio/sample_audio_{}.mp3'.format(recording_num)
sd.wait()
write(recording_name, frame_rate, recording)

print("Finished")

## Play back and plot the user recording 
<pr> User can play back and see a visualization of their latest recording as many times as they like. <pr>

In [ ]:
try: 
    audio, sampling_rate = librosa.load(recording_name)
    
    #plot
    timeline = np.arange(0, len(audio))/sampling_rate
    fig = plt.figure()
    plt.plot(timeline, audio)
    plt.xlabel("Time (s)", fontsize=18)
    plt.ylabel("Amplitude", fontsize=16)
               
    #play back
    sd.play(audio, sampling_rate)
    sd.wait()
    sd.stop()
except(FileNotFoundError):
    print("Please go back and run the previous two cells")


## Delete all user recordings

In [ ]:
try:
    shutil.rmtree(user_audio_directory)
    print("Successfully removed user data")
except(FileNotFoundError):
    print("User data already deleted".format(user_audio_directory)) 

# Classify user recording